In [35]:

import pygame


# User-defined functions

def main():
   # initialize all pygame modules (some need initialization)
   pygame.init()
   # create a pygame display window
   pygame.display.set_mode((500, 400))
   # set the title of the display window
   pygame.display.set_caption('Pong V2')   
   
   w_surface = pygame.display.get_surface() 
   # create a game object
   game = Game(w_surface)
   # start the main game loop by calling the play method on the game object
   game.play() 
   # quit pygame and clean up the pygame window
   pygame.quit() 


# User-defined classes

class Game:
   # An object in this class represents a complete game.

   def __init__(self, surface):
      # Initialize a Game.
      # - self is the Game to initialize
      # - surface is the display window surface object

      # === objects that are part of every game that we will discuss
      self.surface = surface
      self.bg_color = pygame.Color('black')
      #self.score1 = get_score(self)
      self.FPS = 60
      self.game_Clock = pygame.time.Clock()
      self.close_clicked = False
      self.continue_game = True
      
      # === game specific objects
      
      self.mydot = Dot('white', 5, [250,200], [4,1], self.surface)
      self.rect1 = pygame.Rect (100,180,10,40) #left, top, width, height
      self.rect2 = pygame.Rect (400,180,10,40)
      self.paddle1 = Paddle( 'white' , self.rect1, self.surface)
      self.paddle2 = Paddle( 'white', self.rect2, self.surface)
      self.max_frames = 150
      self.frame_counter = 0
      
            
      
   def play(self):
      self.score1 = 0
      self.score2 = 0      
      while not self.close_clicked:
         self.handle_events()
         self.surface.fill(self.bg_color)
         self.draw_score()
         self.draw() 
        
         if self.continue_game == True:    
            self.update()
            self.get_score()
            
            self.decide_continue()
         self.game_Clock.tick(self.FPS) # run at most with FPS Frames Per Second 

   def draw(self):
      # Draw all game objects.
      # - self is the Game to draw
       # clear the display surface first
      self.mydot.draw()
      self.paddle1.draw()
      self.paddle2.draw()
      pygame.display.update() # make the updated surface appear on the display
        
   def draw_score(self):
      self.score_string1 = str(self.score1)
      self.score_string2 = str(self.score2)
      score_fg_color = pygame.Color('white')
      #no bg color needed since we want it to be transparent
      score_font = pygame.font.SysFont('', 50)
      score_image1 = score_font.render(self.score_string1, True, score_fg_color)
      score_image2 = score_font.render(self.score_string2, True, score_fg_color)
      self.surface.blit(score_image1, (10, 10))      
      self.surface.blit(score_image2, (450,10))

   def handle_events(self):
      # Handle each user event by changing the game state appropriately.
      # - self is the Game whose events will be handled

      events = pygame.event.get()
      for event in events:
         if event.type == pygame.QUIT:
            self.close_clicked = True


      
   def move(self,rect1,rect2):
      size = self.surface.get_size()   
      for i in range(len(self.mydot.center)):
         self.mydot.center[i] += self.mydot.velocity[i]
         
         #this bounces mydot off of the edges
         if (self.mydot.center[i] < self.mydot.radius) or ((self.mydot.center[i] + self.mydot.radius) > size[i]):
            self.mydot.velocity[i] = -self.mydot.velocity[i]
        
         #this bounces mydot off of the paddles and through the back side
         if self.rect1[0] <= self.mydot.center[0] <= (self.rect1[0]+18) and self.rect1[1] <= self.mydot.center[1] <= (self.rect1[1]+42) and self.mydot.velocity[0]<0:
            self.mydot.velocity[i] = -self.mydot.velocity[i]
         
         if self.rect2[0] <= self.mydot.center[0] <= (self.rect2[0]+ 22) and self.rect2[1] <= self.mydot.center[1] <= (self.rect2[1]+ 40) and self.mydot.velocity[0]>0:
            self.mydot.velocity[i] = -self.mydot.velocity[i]
            
   def get_score(self):
               
      if self.mydot.center[0] <= 10 and self.mydot.velocity[0] < 0:
         self.score2 += 1
      if self.mydot.center[0] >= 490 and self.mydot.velocity[0] > 0:
         self.score1 +=1 
      #return self.score1
      

      
   def update(self):
      # Update the game objects for the next frame.
      # - self is the Game to update
      
      self.mydot.move()
      self.move(self.rect1,self.rect2)
      self.frame_counter = self.frame_counter + 1

   def decide_continue(self):
      # Check and remember if the game should continue
      # - self is the Game to check
      #When one of the scores reaches 11 end the game
      
      if self.score1==11 or self.score2 == 11:
         self.continue_game = False
      
      


class Dot:
   # An object in this class represents a Dot that moves 
   
   def __init__(self, dot_color, dot_radius, dot_center, dot_velocity, surface):
      # Initialize a Dot.
      # - self is the Dot to initialize
      # - color is the pygame.Color of the mydot
      # - center is a list containing the x and y int
      #   coords of the center of the mydot
      # - radius is the int pixel radius of the mydot
      # - velocity is a list containing the x and y components
      # - surface is the window's pygame.Surface object

      self.color = pygame.Color(dot_color)
      self.radius = dot_radius
      self.center = dot_center
      self.velocity = dot_velocity
      self.surface = surface
      
   def move(self):
      # Change the location of the Dot by adding the corresponding 
      # speed values to the x and y coordinate of its center
      # - self is the Dot
      
      size = self.surface.get_size()   
      for i in range(len(self.center)):
         self.center[i] += self.velocity[i]
         if (self.center[i] < self.radius) or ((self.center[i] + self.radius) > size[i]):
            self.velocity[i] = -self.velocity[i]
         
         #if self.center[0] == self.rect1[0] or self.rect2[0]:
            #self.velocity[i] = -self.velocity[i]
         
      pass
   def draw(self):
      # Draw the mydot on the surface
      # - self is the Dot
      
      pygame.draw.circle(self.surface, self.color, self.center, self.radius)
class Paddle:
   def __init__ ( self, paddle_color, paddle_rect, surface):
      # Initialize paddles.
      # - self is the Paddle to initialize
      # - color is the pygame.Color of the Paddle
      # - width is the integer width of the Paddle
      # - height is the integer height of the Paddle
      # - rect is the location coordinates of the Paddle
      # - surface is the window's pygame.Surface object
      self.color = pygame.Color(paddle_color)
      self.rect = paddle_rect
      self.surface = surface 
      
   def move(self):
      pass
   
   def draw (self):
      #draw the rectangle(Paddle)
      # - self is the Paddle
      pygame.draw.rect(self.surface, self.color , self.rect)

main()